# **Description**
In this notebook you will firstly train a CNN on cifar10 dataset. The CNN architecture you are going to use is ResNet18. You are going to train the model from scratch. Then you are going to implement targeted and untargeted FGSM attack and evaluate the performance of the trained model on the adversarial data.


**Attention!** Make sure you do not miss the questions asked at the end of the notebook.






Name:

Student ID:

#**Setups**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
cd '/content/drive/MyDrive/'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
batch_size = 128
lr = 1e-3
epochs = 100

#**Data Preparation (10 pts)**
Use cifar10 dataset provided in torchvision and create train and test dataloaders according to the given transformations.

In [ ]:
print('==> Preparing data..')

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


###################################################
#prepare training and test datasets and loaders   #
###################################################



######################
# Your code goes here


######################

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

#**Building The Model**

In [ ]:
########################################
#define the model and optimizer        #
########################################

print('==> Building model..')
net = torchvision.models.resnet18(pretrained = False)

device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

optimizer =  optim.SGD(net.parameters(), lr=lr, momentum=0.9)

#**Training and Evaluating The Model (20 pts)**

In [ ]:
#############################################
# define train and test functions           #
#############################################

def train(epoch, net, optimizer, trainloader):
    acc = None
    avg_loss = None
    ######################
    # Your code goes here



    ######################
    return avg_loss, acc


def test(epoch, net, testloader):
    acc = None
    ######################
    # Your code goes here


    ######################
    return acc



In [ ]:
######################################################################
#train your model, make sure to save loss and accuracy in all epochs #
######################################################################

######################
# Your code goes here






#######################

In [ ]:
########################################
#save your model and history           #
########################################


######################
# Your code goes here




#######################

##**Train and Test Loss and Accuracy Plots**

In [ ]:
#############################################################
#plot train and test loss and accuracy dusring the training #
#############################################################

######################
# Your code goes here



######################

# **Visualizing filters (12pts)**

In this section, you're going to visualize filters of some of the layers.

In [ ]:
#########################################################
#Visualize the filters of the specified layers          #
#########################################################


conv_layers = [net.layer1[0].conv1, net.layer1[0].conv2, net.layer2[0].conv1, net.layer2[0].conv2]

##############################
# Your code goes here

##############################



#**Attention Heatmap (8pts)**


Some methods have been proposed to measure how much a model attends to each individual pixel in an image. One of the most well-known methods for this, is [GradCAM](https://github.com/jacobgil/pytorch-grad-cam/), which utilizes the gradient of the model on the last convolutional layers to compute attention scores. Here we want to visualize a number of samples and the model's attention heatmap on them.

In [ ]:
!pip install grad-cam

In [ ]:
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

Select 2 random images from each class in the test set and put them in an array called samples.       

In [ ]:
samples = None
######################
# Your code goes here


 Now we define the heatmap generator. Here we use XGradCAM, which is an extention of the original GradCAM.

In [ ]:
heat_map_generator = XGradCAM(
model=net,
target_layers=[net.layer4[-1]],
)

Visualize the selected images, and the attention heatmap of the model on each of them.

In [ ]:
#############################################################################################
#visualize two test sample from each class and their corresponding attention heatmap        #
#############################################################################################

def visualize_heatmap(image, unnormalized_image, heat_map_generator):
    '''
    inputs:
        image: image which is normalized according to the statistics of the dataset. type: torch.Tensor. shape: [1, channels, width, height]
        unnormalized_image: image that you are going to visualize. the value of its pixels must be in [0,1]. type: numpy array. shape: [width, height, channels]
        heat_map_generator: XGradCAM instance

    returns: visualization of the heatmap on the image
    '''
    grayscale_cam = heat_map_generator(input_tensor=image)
    grayscale_cam = grayscale_cam[0, :]

    vis = show_cam_on_image(unnormalized_image, grayscale_cam, use_rgb=True)
    return vis


######################
# Your code goes here



#**FGSM Attack (50 pts)**
In this section you must implement targeted and untargeted FGSM attack and evalute the trained model performance on the adversarial data.

##**Untargeted FGSM Attack (30 pts)**

In [ ]:
########################################
#define untargeted FGSM attack         #
########################################


def FGSM (model, x, y, epsilon):
  """
  inputs:
    model: model to be attacked
    x: input image
    y: label
    epsilon: clipping threshold for FGSM

  outputs:
    x': the perturbed image
    delta: the added perturbation
    y': prediction of the model for the perturbed input
  """
  delta = None

  ######################
  # Your code goes here




  return x+delta, delta, model(x+delta)

###**Evaluation**


 For $\epsilon \in \{\frac{1}{255}, \frac{4}{255}, \frac{8}{255},\}$ report the performance of the model on targeted FGSM attack on the test data.

In [ ]:
  ######################
  # Your code goes here


###**FGSM by gradually increasing $ϵ$**

FGSM attack may not work for small $\epsilon$, on the other hand we want to alter image as insignificant as possible, so here is what we do, we start from $\epsilon = \frac{1}{255}$ and if the attack was unsuccessful, we increase $\epsilon$ by $\frac{1}{255}$. The procedure goes on until the predicted label is different from the ground truth.

In [ ]:
#######################################################
#increase epsilon gradually till the label is changed. The function can be implemented recursively.#
#######################################################

def generate_adversarial_FGSM (model, x, label, initial_epsilon=1/255):
  """
  inputs:
    model: model to be attacked
    x: input image
    y: label
    epsilon: intial clipping threshold for FGSM

  outputs:
    epsilon: the epsilon that flips the label.
    delta: the added perturbation
    y': prediction of the model for the perturbed input
  """



  ######################
  # Your code goes here


### **Visualization**
Pick 10 samples from cifar10 test set, each from one class, make sure the selected samples are classified correctly. Plot these samples. Take note that the selected data must not be transformed by train or test transformations while visualisation.

In [ ]:
#############################################################################################
#visualize one test sample from each class and make sure that they are correctly labeled.   #
#save index of these samples, since they are needed in the following sections.              #
#############################################################################################






#######################
# Your code goes here



Now, apply untargetted FGSM attack ``` generate_adversarial_FGSM ``` on the selected samples. Visualize the original image, the perturbed image, and the perturbation. since the perturbation is imperceptible, magnify it by the value of $\frac{1}{\epsilon}$ before visualization.

In [ ]:
######################################################################################################
#visualize the selected samples, their corresponding perturbed image, and the magnified perturbation #
#also print the ground truth and the predicted label for each sample                                 #
######################################################################################################




######################
# Your code goes here


##**Targeted FGSM Attack (20pts)**
implement targeted FGSM attack.

In [ ]:
########################################
#define tergeted FGSM attack           #
########################################

def FGSM_targeted (model, x, y, y_targ, epsilon):
  """
  inputs:
    model: model to be attacked
    x: input image
    y: label
    y_targ = target label
    epsilon: clipping threshold for FGSM

  outputs:
    x': the perturbed image
    delta: the added perturbation
  """

  delta = None

  ######################
  # Your code goes here


  return x+delta, delta

###**Evaluation**
for each sample, select the most probable lable as the target label (if the most probable label is the ground truth, select the second most probable label). For $\epsilon \in \{\frac{1}{255}, \frac{4}{255}, \frac{8}{255},\}$ report the performance of the model on targeted FGSM attack on the test data.

In [ ]:
######################
# Your code goes here


for each sample, select the least probable lable as the target label (if the least probable label is the ground truth, select the second most probable label). For $\epsilon \in \{\frac{1}{255}, \frac{4}{255}, \frac{8}{255},\}$ report the performance of the model on targeted FGSM attack on the test data.

In [ ]:
######################
# Your code goes here


plot accuracy of model with respect to $\epsilon$ both for least probable and most probable target labels (both in a single plot).

In [ ]:
######################
# Your code goes here


#**Questions (10 pts)**

1. What was the effect of $\epsilon$ in targeted and untargeted FGSM attack according to your observations?

2. Compare performance of the model on the least probable and most probable targetted FGSM attacks, justify your observation.